# Model Metadata API Demo

The Model Metadata API is still in developement. This demo showcases some of the functionality of the new API.

**Note**: If you are running with self-signed certificate make sure to set `CURL_CA_BUNDLE` environmental variable before executing this notebook, for example start the notebook server with:
```bash
CURL_CA_BUNDLE="" jupyter lab
```

## Api Configuration

In [1]:
from seldon_deploy_sdk.rest import ApiException

from seldon_deploy_sdk import V1Model, ModelMetadataServiceApi, Configuration, ApiClient
from seldon_deploy_sdk.auth import OIDCAuthenticator

In [2]:
from pprint import pprint
import json

In [3]:
# For local testing use
INGRESS = "http://localhost:8080"

# For production usecase set this to your ingress
# for example if your SD URL is http://xxx.yyy.zzz.xyz/seldon-deploy/api/v1alpha1 set
# INGRESS = "http://xxx.yyy.zzz.xyz"

In [4]:
config = Configuration()
config.host = f"{INGRESS}/seldon-deploy/api/v1alpha1"

In [5]:
# auth if needed
config.oidc_server = f"{INGRESS}/auth/realms/deploy-realm"
config.oidc_client_id = "sd-api"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = "client_credentials"

# Alternativey set instead of the "client_credentials" flow
# config.username = "admin@seldon.io"
# config.password = "12341234"
# config.auth_method = "password"

auth = OIDCAuthenticator(config)
config.access_token = auth.authenticate()

In [6]:
api_client = ApiClient(configuration=config, authenticator=auth)
api_instance = ModelMetadataServiceApi(api_client)

## Add single model to the Model Catalogue

In [7]:
model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    artifact_type="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-test-model-alpha-repo",
        "an arbitrary tag": "true",
    },
)
try:
    # Create a Model Metadata entry.
    api_response = api_instance.model_metadata_service_create_model_metadata(model)
except ApiException as e:
    print(f"Couldn't create model: {json.loads(e.body)['message']}")

## Add multiple models to the Model Catalogue

In [8]:
models = [
    #     Same model different versions
    {
        "uri": "gs://test-model-beta-v1.0.0",
        "name": "beta",
        "version": "v1.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Jon"},
    },
    {
        "uri": "gs://test-model-beta-v2.0.0",
        "name": "beta",
        "version": "v2.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
    {
        "uri": "gs://test-model-beta-v3.0.0",
        "name": "beta",
        "version": "v3.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
]

for model in models:
    body = V1Model(**model)
    try:
        api_response = api_instance.model_metadata_service_create_model_metadata(body)
    except ApiException as e:
        print(f"Couldn't create model: {json.loads(e.body)['message']}")

## List all models in the Model Catalogue

In [9]:
try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata()
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'models': [{'artifact_type': 'XGBOOST',
             'creation_time': datetime.datetime(2021, 7, 8, 13, 15, 36, 85891, tzinfo=tzutc()),
             'metrics': {},
             'name': 'alpha',
             'prediction_schema': None,
             'project': 'default',
             'tags': {'an arbitrary tag': 'true',
                      'source': 'https://github.com/some-test-model-alpha-repo'},
             'task_type': 'regression',
             'uri': 'gs://test-model-alpha-v1.0.0',
             'version': 'v1.0.0'},
            {'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2021, 7, 8, 13, 15, 36, 114326, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'prediction_schema': None,
             'project': 'default',
             'tags': {'author': 'Jon'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v1.0.0',
             'version': 'v1.0.0'},
            {'artifact_type': '

## Get all version of a given model (named "beta")

In [10]:
# uri = 'uri_example'                       # str |  (optional)
# name = 'name_example'                     # str |  (optional)
# version = 'version_example'               # str |  (optional)
# artifactType = 'artifactType_example' # str |  (optional)
# task_type = 'task_type_example'           # str |  (optional)
# model_type = 'model_type_example'         # str |  (optional)

try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(name="beta", tags={"author": "Jon"})
    print("Filter by name=beta")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Filter by name=beta
{'models': [{'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2021, 7, 8, 13, 15, 36, 114326, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'prediction_schema': None,
             'project': 'default',
             'tags': {'author': 'Jon'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v1.0.0',
             'version': 'v1.0.0'}],
 'next_page_token': ''}


## Get all models authored by Bob (tags.author = Bob)

In [11]:
# uri = 'uri_example'                       # str |  (optional)
# name = 'name_example'                     # str |  (optional)
# version = 'version_example'               # str |  (optional)
# artifactType = 'artifactType_example' # str |  (optional)
# task_type = 'task_type_example'           # str |  (optional)
# model_type = 'model_type_example'         # str |  (optional)

try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(tags={"author": "Bob"})
    print("Filter by name=beta")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Filter by name=beta
{'models': [{'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2021, 7, 8, 13, 15, 36, 131331, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'prediction_schema': None,
             'project': 'default',
             'tags': {'author': 'Bob'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v2.0.0',
             'version': 'v2.0.0'},
            {'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2021, 7, 8, 13, 15, 36, 150716, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'prediction_schema': None,
             'project': 'default',
             'tags': {'author': 'Bob'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v3.0.0',
             'version': 'v3.0.0'}],
 'next_page_token': ''}


## Modify model metadata entry in the Model Catalogue

In [12]:
try:
    # Get Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(uri="gs://test-model-alpha-v1.0.0")
    print("Before update:")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")


model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    artifact_type="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-other-repo",
        "an arbitrary tag": "true",
        "an additional tag": "123",
    },
)

try:
    # Update a Model Metadata entry.
    api_response = api_instance.model_metadata_service_update_model_metadata(model)
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")
    
try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(
        uri="gs://test-model-alpha-v1.0.0"
    )
    print("After update:")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Before update:
{'models': [{'artifact_type': 'XGBOOST',
             'creation_time': datetime.datetime(2021, 7, 8, 13, 15, 36, 85891, tzinfo=tzutc()),
             'metrics': {},
             'name': 'alpha',
             'prediction_schema': None,
             'project': 'default',
             'tags': {'an arbitrary tag': 'true',
                      'source': 'https://github.com/some-test-model-alpha-repo'},
             'task_type': 'regression',
             'uri': 'gs://test-model-alpha-v1.0.0',
             'version': 'v1.0.0'}],
 'next_page_token': ''}
Failed to call API: code=NotFound desc=record not found
After update:
{'models': [{'artifact_type': 'XGBOOST',
             'creation_time': datetime.datetime(2021, 7, 8, 13, 15, 36, 85891, tzinfo=tzutc()),
             'metrics': {},
             'name': 'alpha',
             'prediction_schema': None,
             'project': 'default',
             'tags': {'an arbitrary tag': 'true',
                      'source': 'https://

## Get runtime information for a model

In [13]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = api_instance.model_metadata_service_list_runtime_metadata_for_model(
        model_uri="gs://seldon-models/sklearn/iris", 
        deployment_status="Running"
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'next_page_token': '',
 'runtime_metadata': [{'creation_time': datetime.datetime(2021, 7, 8, 12, 44, 37, 29639, tzinfo=tzutc()),
                       'deployment_kubernetes_uid': 'e26d3103-4f84-4044-97ef-4d704b67bd75',
                       'deployment_name': 'iris-canary',
                       'deployment_namespace': 'seldon-demos',
                       'deployment_status': 'Running',
                       'deployment_type': 'SeldonDeployment',
                       'model': {'artifact_type': 'SKLEARN',
                                 'creation_time': datetime.datetime(2021, 7, 8, 12, 44, 37, 25240, tzinfo=tzutc()),
                                 'metrics': {},
                                 'name': '',
                                 'prediction_schema': None,
                                 'project': 'default',
                                 'tags': {'auto_created': 'true'},
                                 'task_type': '',
                                 'uri':

## Get model information for a deployment

In [14]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = api_instance.model_metadata_service_list_runtime_metadata_for_model(
        deployment_name="iris", 
        deployment_namespace="seldon"
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'next_page_token': '',
 'runtime_metadata': [{'creation_time': datetime.datetime(2021, 7, 8, 12, 48, 49, 996794, tzinfo=tzutc()),
                       'deployment_kubernetes_uid': '9d09ed08-21ca-4821-96d5-12b9ebed7f13',
                       'deployment_name': 'iris',
                       'deployment_namespace': 'seldon',
                       'deployment_status': 'Running',
                       'deployment_type': 'SeldonDeployment',
                       'model': {'artifact_type': 'SKLEARN',
                                 'creation_time': datetime.datetime(2021, 7, 8, 12, 44, 37, 25240, tzinfo=tzutc()),
                                 'metrics': {},
                                 'name': '',
                                 'prediction_schema': None,
                                 'project': 'default',
                                 'tags': {'auto_created': 'true'},
                                 'task_type': '',
                                 'uri': 'gs://seldo